Daftlistings library enables programmatic interaction with Daft.ie. Daft.ie has nationwide coverage and contains about 80% of the total available properties in Ireland.
* https://github.com/AnthonyBloomer/daftlistings
* pip install https://github.com/AnthonyBloomer/daftlistings/archive/dev.zip

In [1]:
import pandas as pd
import csv
from daftlistings import Daft, Location, SearchType, PropertyType, Facility
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### Extract properties are listed under RESIDENTIAL_SALE

In [9]:
# daft = Daft()
# daft.set_search_type(SearchType.RESIDENTIAL_SALE)

# listings = daft.search(max_pages=100)

# # cache the listings in the local file
# with open("DaftResidentialSale.txt", "w") as fp:
#     fp.writelines("%s\n" % listing.as_dict() for listing in listings)

# read from the local file
with open("./files/DaftResidentialSale.txt") as fp:
  lines = fp.readlines()

properties = []
for line in lines:
  properties.append(eval(line))

DaftResidentialSale = pd.DataFrame(properties)

### Extract properties are listed under New Homes

In [10]:
# daft = Daft()
# daft.set_search_type(SearchType.NEW_HOMES)

# listings = daft.search()

# # cache the listings in the local file
# with open("DaftNewHomes.txt", "w") as fp:
#     fp.writelines("%s\n" % listing.as_dict() for listing in listings)

# read from the local file
with open("./files/DaftNewHomes.txt") as fp:
  lines = fp.readlines()

properties = []
for line in lines:
  properties.append(eval(line))

DaftNewHomes = pd.DataFrame(properties)

### Check duplications in ResidentialSale and NewHomes

There are 7 search types according to the class SearchType(enum.Enum) in the Daftlisting library. We are interested in the residential for sale, but it is odd new homes are in a seperated category. Double chekck if residential sale and new homes do not have intersection. After eveluating duplications, merge two dataframe to have a full picture of residential sale properties.

* RESIDENTIAL_SALE = "residential-for-sale"
* RESIDENTIAL_RENT = "residential-to-rent"
* COMMERCIAL_SALE = "commercial-for-sale"
* COMMERCIAL_RENT = "commercial-for-rent"
* SHARING = "sharing"
* STUDENT_ACCOMMODATION = "student-accommodation-to-share"
* NEW_HOMES = "new-homes"
##### Remove Duplications #####
1. Residential Sale data has duplications, remove duplicated rows after the evaluation.
2. Same ids in residential sales and new homes are the same properties after checking "comparison.csv". It is safe to remove the duplications and concat two data frames. 

In [11]:
# Evaluate the duplicated rows in DaftResidentialSale
duplicated_rows = DaftResidentialSale.duplicated(subset=['id'], keep=False)
DaftResidentialSale[duplicated_rows].sort_values(by="id")

# Drop duplications in DaftResidentialSale
DaftResidentialSale.drop_duplicates(subset=['id'], inplace=True)

In [12]:
# Check if same ids in both DaftResidentialSale and DaftNewHomes are the same properties
id1 = set(list(DaftResidentialSale.id))
id2 = set(list(DaftNewHomes.id))
same_ids = id1.intersection(id2)
df1 = DaftResidentialSale[DaftResidentialSale.id.isin(same_ids)].sort_values(by="id", ascending=False)
df1["Flag"] = 1
df2 = DaftNewHomes[DaftNewHomes.id.isin(same_ids)].sort_values(by="id", ascending=False)
df2["Flag"] = 2

df3 = pd.concat([df1, df2]).sort_values(by="id", ascending=False)
df3.to_csv("./files/comparison.csv") #check the comparison.csv, will find same ids mean same properties.


In [13]:
# Checkign data shape
# print(DaftResidentialSale.shape)
# print(DaftNewHomes.shape)
# print(len(same_ids))
# print(DaftResidentialSale.shape[0]+DaftNewHomes.shape[0]-len(same_ids))

# Concat two dataframes, and drop the duplications
data = pd.concat([DaftResidentialSale, DaftNewHomes], ignore_index=True)
data.drop_duplicates(subset=['id'], inplace=True)

# export to a csv file
#all residential listings in Ireland that listed on Daft.ie (date: 30/01/2022)
data.to_csv("./files/Daft_residential_listings.csv", index=False) #(5310, 31) 


# Filtering Sligo data only

In [14]:
# from daftlistings import Daft, Location, SearchType, PropertyType

# # daft = Daft()
# # daft.set_location(Location.SLIGO)
# # daft.set_search_type(SearchType.RESIDENTIAL_SALE)

# # listings = daft.search()


# # # cache the listings in the local file
# # with open("DaftSligoHomes.txt", "w") as fp:
# #     fp.writelines("%s\n" % listing.as_dict() for listing in listings)

# # read from the local file
# with open("./files/DaftSligoHomes.txt") as fp:
#   lines = fp.readlines()

# properties = []
# for line in lines:
#   properties.append(eval(line))

# # export to a csv file
# #all residential listings in Sligo, Ireland that listed on Daft.ie (date: 30/01/2022)
# DaftSligoHomes = pd.DataFrame(properties)
# DaftSligoHomes.to_csv("./files/Daft_sligo_residential_listings.csv", index=False) #(132, 26)

In [2]:
import pandas as pd
from daftlistings import Daft, Location, SearchType, PropertyType, SortType, MapVisualization

 
daft = Daft()
daft.set_location(Location.SLIGO)
daft.set_search_type(SearchType.RESIDENTIAL_SALE)
daft.set_sort_type(SortType.PRICE_ASC)


listings = daft.search()

# cache the listings in the local file
with open("DaftSligoHomes.txt", "w") as fp:
    fp.writelines("%s\n" % listing.as_dict_for_mapping() for listing in listings)

# read from the local file
with open("DaftSligoHomes.txt") as fp:
  lines = fp.readlines()

properties = []
for line in lines:
  properties.append(eval(line))


# export to a csv file
#all residential listings in Sligo, Ireland that listed on Daft.ie (date: 06/02/2022)
DaftSligoHomes = pd.DataFrame(properties)
DaftSligoHomes.to_csv("./files/Daft_sligo_residential_listings.csv", index=False) 

Searching...
Search complete. Found 175 listings.
